In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Activation,Input,BatchNormalization,Flatten, Dense, Conv2D, MaxPool2D,GlobalAveragePooling2D,Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
class BottleNeck(tf.keras.layers.Layer):

    def __init__(self, num_filters, stride):
        super(BottleNeck, self).__init__()

        self.conv1 = Conv2D(filters = num_filters, kernel_size=(3,3), strides= (1,1), padding = 'same')

        self.bn1 = BatchNormalization()

        

        self.conv2 = Conv2D(filters = num_filters, kernel_size=(3,3), strides= stride, padding = 'same')

        self.bn2 = BatchNormalization()
        

        self.shortcut = Sequential()

        self.shortcut.add(Conv2D(num_filters, kernel_size=(3,3), strides= stride, padding = 'same'))

        self.shortcut.add(BatchNormalization())

            

    def call(self, inputs, training=None, **kwargs):

        

        x = self.conv1(inputs)

        x = tf.nn.relu(self.bn1(x))

        x = self.conv2(x)

        x = tf.nn.relu(self.bn2(x))

        output = tf.nn.relu(tf.keras.layers.add([self.shortcut(inputs), x]))

        

        return output


In [3]:
def _make_layer(num_filters, num_blocks, stride):

    

    res_block = Sequential()

    res_block.add(BottleNeck(num_filters, stride=stride))

    

    for _ in range(1, num_blocks): # 1에서 시작해서 num_block 수만큼 반복

        res_block.add(BottleNeck(num_filters, stride=1))

    

    return res_block


In [4]:
#class 선언하면 위에 import한 거와는 별개로 전혀 연동이 안 되는 듯...
"""
Model: combines various layers to build a machine learning model
Model이 있어야 compile, fit, evaluate, predict 등의 함수를 이용할 수 있다.
"""

class ResNet(tf.keras.Model):

    def __init__(self, num_block, num_units=10):
        super(ResNet,self).__init__()

        

        self.conv1 = Conv2D(filters=64,kernel_size=(7,7),strides=(2,2),padding='same')

        self.bn1 = BatchNormalization()

        

        self.mp1 = MaxPool2D(pool_size = (3,3), strides=(2,2), padding='same')

        self.bn2 = BatchNormalization()


        self.bt1 = _make_layer(16, num_block[0], 1)

        self.bt2 = _make_layer(32, num_block[1], 2)

        self.bt3 = _make_layer(64, num_block[2], 2)

        

        self.av1 = GlobalAveragePooling2D()

        self.flatten = Flatten()

        self.ds1 = Dense(units=256,activation='relu')

        self.ds2 = Dense(units=128,activation='relu')

        self.ds3 = Dense(units=num_units,activation='softmax')

        

    def call(self, inputs, training=None, mask=None):

        x = self.conv1(inputs)

        x = self.bn1(x, training=training)

        x = tf.nn.relu(x)

        x = self.mp1(x)

        x = self.bn2(x)

        x = self.bt1(x, training=training)

        x = self.bt2(x, training=training)

        x = self.bt3(x, training=training)

        x = self.av1(x)

        x = self.flatten(x)

        x = self.ds1(x)

        x = self.ds2(x)

        output = self.ds3(x)


        return output


def ResNet34():

    return ResNet(num_block = [3,4,6,3])


In [5]:
def get_model():

    model = ResNet34()

    model.build(input_shape=(None,32,32,3))

    model.summary()

    return model


model = get_model()


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full outp

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  9472      
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
____________________

In [6]:
def load_cifar10_dataset():
    (train_validation_images, train_validation_labels), (test_images, test_labels) \
    = cifar10.load_data()
    
    train_validation_images = train_validation_images / 255
    test_images = test_images / 255.
    
    n_train_validation = train_validation_images.shape[0]
    train_ratio = 0.8
    n_train = int(n_train_validation * train_ratio)
    
    train_x = train_validation_images[:n_train]
    train_y = train_validation_labels[:n_train]
    validation_x = train_validation_images[n_train:]
    validation_y = train_validation_labels[n_train:]
    test_x = test_images
    test_y = test_labels
    
    print('Train set')
    print('Image :', train_x.shape)
    print('Label :', train_y.shape)
    print('Validation set')
    print('Image :', validation_x.shape)
    print('Label :', validation_y.shape)
    print('Train & validation set')
    print('Image :', test_x.shape)
    print('Label :', test_y.shape)
    
    return train_x, train_y, validation_x, validation_y, test_x, test_y

In [7]:
train_x, train_y, validation_x, validation_y, test_x, test_y = load_cifar10_dataset()


train_y = tf.keras.utils.to_categorical(train_y,10)

validation_y = tf.keras.utils.to_categorical(validation_y,10)

test_y = tf.keras.utils.to_categorical(test_y,10)

Train set
Image : (40000, 32, 32, 3)
Label : (40000, 1)
Validation set
Image : (10000, 32, 32, 3)
Label : (10000, 1)
Train & validation set
Image : (10000, 32, 32, 3)
Label : (10000, 1)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01),loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(train_x, train_y, epochs=50, batch_size=32,validation_data=(validation_x, validation_y))

ERROR! Session/line number was not unique in database. History logging moved to new session 109
Train on 40000 samples, validate on 10000 samples
Epoch 1/50
   32/40000 [..............................] - ETA: 1:13:03

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tlsdu\AppData\Local\Temp\ipykernel_12152\567259730.py", line 2, in <module>
    history = model.fit(train_x, train_y, epochs=50, batch_size=32,validation_data=(validation_x, validation_y))
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 342, in fit
    total_epochs=epochs)
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 128, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\tensorflo